In [ ]:
import nibabel as nb
import numpy as np
import pandas as pd 
from nilearn import plotting
from fragmenter import Fragment
from fragmenter import adjacency

%matplotlib inline
import matplotlib.pyplot as plt

This is a simple demo of the parcellation fragmenter, from simple use to some benchmarks. The first step is to load a template surface (in this case, a random participant).

# Vision

 *  Examine connectivity metrics as a function of cortical map resolution (# clusters)
     * For example, iteratively sub-parcellate exising regions and examine if metrics break down, plateau etc.
     
 * Use tractography or correlation profiles of sub-regions as higher-resolution features in machine learning algorithms
     * Caveat: requires preprocessing step to "match" parcels across subjects (Hungarian algorithm)
     * http://www.math.harvard.edu/archive/20_spring_05/handouts/assignment_overheads.pdf
     
     
 * Decrease processing time for some tools
     * Searchlight FDR -- rather than searching over all vertices, search over parcels instead
     * Modularity on 5000 parcels versus 32,000 vertices

# Demo

Surfaces are a 2D triangular mesh. When loading a surface, you get a tuple of two arrays: the 3D-space coordinates of each vertex and a list of triangles that represent the surface, such that each triangle is composed of 3 vertices.

The FreeSurfer mesh we're using here has ~164K vertices (fsaverage). Note that this package also accepts other standards, such as the Human Connectome Project's surface.

In [ ]:
vertices,faces = nb.freesurfer.io.read_geometry('../data/freesurfer/fsaverage/surf/lh.sphere')

In [ ]:
print('n_vertices: {:}'.format(vertices.shape[0]))

In order to parcellate a surface, we derive the neighbors of each vertex. You can consider this to be an adjacency matrix that allows for standard clustering methods to be computed on it. The `adjacency` function will give you this list (in this example, we will just look at the neighbors of vertex 2).

In [ ]:
# Create a surface adjacency object
M = adjacency.SurfaceAdjacency(vertices=vertices, faces=faces)

# Generate adjacency list
M.generate()

# Visualize
print('Neighbors of vertex 1: ')
print(M.adj[1])

In order to fragment a surface, we first create a `Fragment` object that takes in the desired number of clusters, and whether you want to use pretty colors (True/False). In this case, we will run a simple 10 parcel example.

In [ ]:
whole_brain = Fragment.Fragment(n_clusters=75, use_pretty_colors=False)

Once the Fragment object is instantiated, you can see that it now has an attribute called `n_clusters` containing the number of clusters to generate.

In [ ]:
print('Numer of clusters to generate: {:}'.format(whole_brain.n_clusters))

We wrote the Fragment class to follow the sklearn format with methods like `fit` and `predict`.  We're going to fit the fragmentation method, feeding in the vertices and faces.  The following example uses k-means clustering for parcellating our surface.

In [ ]:
whole_brain.fit(vertices = vertices, faces=faces, method = 'k_means')

This will add an attribute to `whole_brain` called `label_`, which is the fragmented cortical map, where each vertex is now assigned a new label value. We can confirm this by checking that length of the resulting vector is equal to the number of vertices in fsaverage.

In [ ]:
print('# Generated Clusters: {:}'.format(np.unique(whole_brain.label_).shape))

And now we can plot the 75 parcels onto a surface using Nilearn's ```view_surf``` method.

In [ ]:
inflated_vertices, inflated_faces = nb.freesurfer.io.read_geometry(
        '../data/freesurfer/fsaverage/surf/lh.inflated')
plotting.view_surf([inflated_vertices,inflated_faces], whole_brain.label_,threshold=0.1,cmap='jet')

Parcellating the whole brain is haphazard, and doesn't respect any previous anatomical boundaries.  We wanted to be able to parcellate individual regions as well. To do this, we first extract the vertices associated with each region using the `RegionExtractor` class -- we represent this output as a dictionary structure mapping region names to region indices -- we can then feed this dictionary back into out `Fragment` object, along with a list of ROIs we're interested in parcellating

Let's specifically parcellate the temporal lobe and inferior parietal region, respecting the original boundaries by the the Desikan-Killiany atlas.

In [ ]:
from fragmenter import RegionExtractor

# Load atlas
label_file = '../data/freesurfer/fsaverage/label/lh.aparc.annot'

# Extract the vertices associated with each region, pick areas to parcellate
E = RegionExtractor.Extractor(label_file)
parcels = E.map_regions()

# Define a list of regions of interest
rois=['temporalpole','inferiortemporal','middletemporal','superiortemporal',
      'transversetemporal','bankssts',
      'inferiorparietal','supramarginal']

We fit the model again, this time proving the `parcels` dictionary and `rois` list.  This time, let's specifiy that we want each generated parcel to have a specific `size` -- in this case, defined by the number of vertices.  If you know the length of each edge in *mm*, you can length parameterize fragment size by surface area, to make more biologically meaningful sub-regions (note that this option overrides `n_clusters`).

In [ ]:
temporal = Fragment.Fragment(n_clusters=75)
temporal.fit(vertices=vertices,faces=faces,parcels=parcels,rois=rois,size=15)

Now, if we plot this new label map, we'll see that we have independently fragmented each of the specified Desikan Killiany Regions:

In [ ]:
plotting.view_surf([inflated_vertices,inflated_faces],temporal.label_,threshold=0.1,cmap='plasma')

Now, if we want to use a pretty colormap to plot these new labels, we can do the following:

In [ ]:
from fragmenter import colormaps

In [ ]:
[keys, ctab, names, remapped] = colormaps.get_ctab_and_names(vertices,temporal.label_)
plotting.view_surf([inflated_vertices,inflated_faces], remapped, threshold=0.1,cmap='plasma')

Note that the sub-region colors then follow a gradient determined by the superior-inferior axis.  If we want to save the generated map, we can save the label to a FreeSurfer annotation file with:

In [ ]:
annot_name = '../data/freesurfer/temporal.annot'
temporal.write(annot_name, use_pretty_colors=True)

Alternatively, if you want to save the label maps to export and use independently (say, in HCP's workbench, or in R), you can simply write them to a csv file. This can be useful if you are reducing the dimensionality of the cortex in order to run community detection algorithms in igraph (R version).

In [ ]:
# Save for external use
np.savetxt("WBLabels.csv", whole_brain.label_, '%5.0f')

If you were fragmenting the HCP surface (e.g. 32k), you can produce a CIFTI dscalar.nii file to view in the workbench using the wb_command utilities. For example:

``` {bash}
wb_command -cifti-convert -from-text WBLabels.csv 100307.MyelinMap_BC.32k_fs_LR.dscalar.nii WBLabels.dscalar.nii
```

Keep in mind that the `cifti-convert` function requires an example file to generate the new one (in this case, 100307's 32k myelin map).

On top of the base fragmentation code, we've also developed some methods to generate "null" parcellations. These produce slight shifts of the location of the labels either through an affine rotation or probabilistic remapping of edges. If you would like to learn more about the algorithms used to fraction the surfaces, see the benchmark notebook for details.

# Conclusions

 *  Developed a fast way to generate new cortical maps for use in null models, and higher-resolution models
 *  Could easily fit into typical connectivity analysis pipelines for generating regional connectivity matrices

# Things We Learned

 *  **TONS** of Git -- hopefully you never find youself in a situation where your local clone and repo histories don't align (thanks Kirstie for the help!!)
 *  moving from R to Python
 *  writing object-oriented code
 *  properly packaging and distributing software
 *  collaborative coding -- hugely beneficial -- we all felt that this was invaluable to improving our own programming skills, question our biases for how to go about solving problems
 *  ever-so-slight inspiration for how to use Javascript

# Thanks to
 *  Ariel, Tal, and Kirstie
 *  Michael and Ross for the inspiration and Git help